<a href="https://colab.research.google.com/github/databyhuseyn/DeepLearning/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d ishantjuyal/emotions-in-text

Dataset URL: https://www.kaggle.com/datasets/ishantjuyal/emotions-in-text
License(s): copyright-authors
emotions-in-text.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!wget https://raw.githubusercontent.com/databyhuseyn/DeepLearning/main/helper_functions.py

--2024-05-27 10:55:59--  https://raw.githubusercontent.com/databyhuseyn/DeepLearning/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-05-27 10:56:00 (29.4 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [ ]:
from helper_functions import unzip_data

In [ ]:
unzip_data('/content/emotions-in-text.zip')

In [ ]:
data = pd.read_csv('/content/Emotion_final.csv')
data

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
21454,Melissa stared at her friend in dism,fear
21455,Successive state elections have seen the gover...,fear
21456,Vincent was irritated but not dismay,fear
21457,Kendall-Hume turned back to face the dismayed ...,fear


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21459 entries, 0 to 21458
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     21459 non-null  object
 1   Emotion  21459 non-null  object
dtypes: object(2)
memory usage: 335.4+ KB


In [ ]:
data.describe()

,Text,Emotion
count,21459,21459
unique,21405,6
top,i feel so tortured by it,happy
freq,2,7029


In [ ]:
df_shuffled = data.sample(frac=1, random_state=42)
df_shuffled

,Text,Emotion
20248,"There was amusement in Jake 's voice , but the...",happy
8094,i feel it is too dangerous to invest in such m...,anger
2859,i feel that i should write the company and tel...,sadness
18953,i also feel ashamed at the hurt caused and ash...,sadness
3931,i just yearned for that homey feeling where yo...,love
...,...,...
11284,i want to savor this feeling of ecstatic antic...,happy
11964,im feeling all puppy dogs and rainbows when im...,sadness
5390,i feel is very delicate,love
860,i was starting to feel a little stressed,sadness


In [ ]:
X = df_shuffled['Text']
y = df_shuffled['Emotion']

In [ ]:
y.value_counts()

Emotion
happy       7029
sadness     6265
anger       2993
fear        2652
love        1641
surprise     879
Name: count, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

onehot = OneHotEncoder(sparse=False)

y_train = onehot.fit_transform(y_train.values.reshape(-1, 1))
y_test = onehot.transform(y_test.values.reshape(-1, 1))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
X_test.head()

16848    im saying this having not read the book the ch...
17413      i think ive just been feeling a little bothered
13593    i don t feel special and when i feel alone in ...
12430    ive missed over a month of training and organi...
869      i feel like i should go to the supermarket and...
Name: Text, dtype: object

In [ ]:
max_length = round(sum([len(i.split()) for i in X_train]) / len(X_train))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

max_vocab_length = 10_000
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [ ]:
text_vectorizer.adapt(X_train)

In [ ]:
sample_sentence = "He lost his grandma"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 19), dtype=int64, numpy=
array([[ 44, 278,  74,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0]])>

In [ ]:
tf.random.set_seed(42)
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer='uniform',
                             input_length=max_length,
                             name='embedding')

# Model 0:

In [ ]:
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, activation='relu', kernel_regularizer='l2')(x)
outputs = layers.Dense(len(data['Emotion'].value_counts()), activation='sigmoid')(x)
model_0 = tf.keras.Model(inputs, outputs, name='model_0')

In [ ]:
model_0.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics='accuracy')

In [ ]:
model_0.summary()

Model: "model_0"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 19)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 19, 128)           1280000   
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 6)                 390       
                                                                 
Total params: 1329798 (5.07 MB)
Trainable params: 1329798 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [ ]:
model_0_history = model_0.fit(X_train,
                              y_train,
                              epochs=5,
                              validation_data=(X_test, y_test))

Epoch 1/5
537/537 [==============================] - 33s 50ms/step - loss: 1.6858 - accuracy: 0.3147 - val_loss: 1.6744 - val_accuracy: 0.3276
Epoch 2/5
537/537 [==============================] - 17s 32ms/step - loss: 1.3810 - accuracy: 0.3914 - val_loss: 1.3568 - val_accuracy: 0.4993
Epoch 3/5
537/537 [==============================] - 18s 33ms/step - loss: 0.9344 - accuracy: 0.6434 - val_loss: 0.9181 - val_accuracy: 0.6983
Epoch 4/5
537/537 [==============================] - 18s 34ms/step - loss: 0.6823 - accuracy: 0.7629 - val_loss: 0.8044 - val_accuracy: 0.7446
Epoch 5/5
537/537 [==============================] - 18s 34ms/step - loss: 0.5270 - accuracy: 0.8346 - val_loss: 0.7882 - val_accuracy: 0.7756


In [ ]:
prediction_position = np.argmax(model_0.predict(['She lost her mum']))
prediction_position

1/1 [==============================] - 0s 241ms/step


4

In [ ]:
model_0.predict(['She lost her mum'])

1/1 [==============================] - 0s 63ms/step


array([[0.4002287 , 0.2241281 , 0.5926574 , 0.13963908, 0.8951469 ,
        0.03130883]], dtype=float32)

In [ ]:
models_prediction = onehot.get_feature_names_out()[prediction_position][3:]
models_prediction

'sadness'